In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import math
import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
from transliterate import translit, get_available_language_codes
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale, MaxAbsScaler, RobustScaler, QuantileTransformer, Normalizer
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
sns.set()
%matplotlib inline
pd.set_option('display.max_colwidth', 1500)
from sklearn.feature_selection import SelectKBest, chi2
from scipy import sparse
from scipy.sparse import csr_matrix
from nltk.tokenize import MWETokenizer
from nltk.stem.snowball import RussianStemmer

In [ ]:
cols = ['resume_id','area_id','position','gender','birth_day','education_level','compensation','currency',
        'relocation','visibility','lang','status','creation_time','last_change_time','source','road_time',
        'business_trip_readiness','work_schedule','employment','disabled',
        'work_experience_month_count','professional_area','skills','resume_id.1',
        'kir','mya','udm','aze','sqi','ara','hye','afr','bak','bel','bul','hun','vie','nld','ell','kat','dan','heb','spa','ita','kaz','zho','kom','kor','kur','lav','lit','mke','mon','nor','fas','pol','por','ron','rus','san','srp','slk','slv','swa','tgk','tha','tat','tur','tuk','uzb','ukr','urd','fin','hrv','che','ces','chv','swe','est','jpn','hin','eng','deu','fra','ind','inh','kas','epo','khm','crs','abk','oss','kbd','krc','cat','uig','abq','kum','tam','lat','zlm','nog','amh','dar','ava','eus','ben','bos','bua','dag','gle','isl','krl','lbe','lao','lez','chm','nep','pan','pus','som','tgl','tly','bod',
        'spec_name']
resumes0 = pd.read_csv('/home/shurik2533/all_resumes.csv', usecols=cols)

In [ ]:
resumes0['spec_name_hash'] = resumes0['spec_name'].apply(lambda x: translit(x, 'ru', reversed=True))

def spc(s):
    sa = s.split('; ')
    sa.sort()
    return '; '.join(sa)
resumes0['spec_name'] = resumes0.spec_name.apply(spc)

In [ ]:
pa = resumes0.professional_area.value_counts()
pa_names = pa.index
pa_translits = [translit(re.sub("\W+", '', v), 'ru', reversed=True) for v in pa_names]
use_skills = [v == 'Информационные технологии, интернет, телеком' for v in pa_names]
pa

In [ ]:
m = Mystem()
tknzr = MWETokenizer()
stemmer = RussianStemmer()
def tokenize(text):
    return tknzr.tokenize([stemmer.stem(v) for v in re.sub(',', '', text.lower()).split() if len(stemmer.stem(v)) > 0])

def tokenize_skills(text):
    text = text.replace(r"'\N'", ' ')
    skills = text.lower().split(', ')
    res = []
    for skill in skills:
        skill_l = [stemmer.stem(l) for l in tknzr.tokenize(skill.split()) if l not in stop_words]
        skill_l.sort()
        skill = ' '.join(skill_l)
        if len(skill) > 1:
            res.append(skill)
            res = res + skill.split()
    return set(res)

def tokenize_spec(text):
    return [x.strip() for x in text.split(';')]

bad_words = 'знание саморазвитие напористость владение наличие быстро принимать решение самостоятельно умение жизнерадостность проявлять тактичность умение межличностный навык общение организовывать работа умение быстро учиться убеждать умение опыт работа проявлять тактичность толерантность умение перфекционист адекватность инициативный добропорядочный дисциплинировать аккуратный порядочный холостой отзывчивый добрый стараться and способность двое мама другой ответственный опыт нравиться как офис выбирать профессия надежность честность проведение навык должность этот быть замужем уравновешивать как желающий хотеть именно общительный целеустремленный задача адекватный пунктуальный опыт стрессоустойчивый ум коммуникабельный тактичный трудолюбивый терпеливый искать срочно приятный внешний увлекаться выполнять своевременно В последние годы проходил проходила обучение без возможности работать, Грамотная речь, презентабельная внешность , коммуникабельность, ответственность‚ исполнительность, многозадачность, стрессоустойчивость, обучаемость, внимательность, аккуратность, доброжелательность, умение работать в команде, Отсутствие вредных привычек, энергичность, инициативность, самостоятельность, ответственность, коммуникабельность, быстрая обучаемость, Имею необходимый стаж работы по специальности  Резюме создано    для   на работе, Ответственность, нацеленность на результат, стрессоустойчивость, стремление к развитию и росту, коммуникабельность, обучаемость, ответственность, трудолюбие, умение располагать к себе. В последние годы проходил службу в армии, Коммуникабельность, целеустремленность , активность‚ доброжелательность‚ аккуратность, обучаемость, стремление повышать свою квалификацию, желание зарабатывать, ответственность.  Грамотная речь, коммуникабельность, ответственность‚ исполнительность, многозадачность, стрессоустойчивость, обучаемость, внимательность, аккуратность, доброжелательность, умение работать в команде.  В последнее время нахожусь в поисках работы. Внимательность‚ ответственность‚ аккуратность, усидчивость, умение быстро адаптироваться в новом коллективе, стрессоустойчивость, исполнительность, обучаемость. Отзывчивость  Вежливость  Умение легко находить контакт с людьми  Уверенность в себе  Умение легко поддерживать деловые и профессиональные контакты  Умение расставлять приоритеты  Оптимизм  Активная жизненная позиция  Умение добиваться поставленных целей  Умение эффективно планировать свое время  Креативное мышление  Системное мышление  Внимательность к деталям  Готовность самостоятельно принимать решения  Умение работать в команде  Стремление к профессиональному развитию  Инициативность  Ответственность, порядочность, умение работать в команде В последние годы находилась в декретном отпуске.  Хобби Пунктуальна, коммуникабельна и очень ответственна, умею работать в команде, быстро обучаюсь. оммуникабельность, активная жизненная позиция, нацеленность на результат, стрессоустойчивость, стремление к развитию и профессиональному росту, исполнительность, пунктуальность, честность Ответственный подход к трудовой деятельности,Умение легко находить контакт с людьми,отзывчива и вежлива Аналитический склад ума,пунктуальная,коммуникабельная, не конфликтная,легко обучаемая,ответственная,внимательная,умение работать Не боюсь ответственности, исполнительна, инициативна, организованна, умею работать - Ответственный - Трудолюбивый - Отзывчивый - Стрессоустойчивый - Целеустремленный - Истиный жизнелюб - В меру авантюрист - Активный - Позитивный - Коммуникабельный - Честный Грамотное‚ четкое и своевременное выполнение поставленных задач, ответственное отношение к работе; - Готовность работать в режиме многозадачности; - Организованность, умение расставлять приоритеты при решении различных рабочих задач; - Легко и с интересом изучаю новые области и технологии; - В своей работе стремлюсь к развитию и повышению уровня профессиональных компетенций. Ответственность, добросовестность, исполнительность, внимательность, высокая работоспособность, коммуникабельность, стрессоустойчивость Активная жизненная позиция, умею работать в команде, коммуникабельна, всегда нацелена на результат. Стрессоустойчива, стремлюсь к развитию и профессиональному росту. По жизни самостоятельна. Личные качества: ответственность, стрессоустойчивость, аналитический склад ума, целеустремленность, быстрая обучаемость.  проактивность В жизни и работе не равнодушен к проблемам людей которые обращаются ко мне за помощью. Люблю активные виды спорта, вредных привычек не имею. Диплом лучшего сотрудника отделения Целеустремленная, могу работать в условиях многозадачности, стрессоустойчивая, коммуникабельная. Я Трудолюбив , тактичен ,пунктуален, вынослив и стрессоустойчив, позитивный и стрессоустойчив , могу наладить контакт с любым человеком, найду выход из сложной ситуации, могу найти индивидуальный подход к каждой ситуации Усидчивость, настойчивость, коммуникабельность,обучаемость... Нужна работа.свою работу нужно любить иначе ничего не получится Личные качества: исполнительность, коммуникабельность,инициативность, ответственность, целеустремленность, активность, нацеленность на результат, организованность, обучаемость. Высшее образование с красным дипломом. Пунктуальность, целеустремленность, стрессоустойчивость, умение устанавливать контакт с людьми, трудолюбие, стремление и умение достигать поставленной цели, работать на результат, настойчивость, обладание большими Амбициями, честность, дружелюбие, ответственность. Пунктуальный, целеустремленный, усидчивый, вежливый. Ищу только постоянную работу !!! Творческие мышление, целеустремленность, коммуникабельность, пунктуальность, трудолюбие, ответственность, нацеленность на результат, активная жизненная позиция, аналитический склад ума, исполнительность. Целеустремленный, ответственный, коммуникабельный, внимательный, стрессоустойчив, легко обучаем; обладаю грамотной речью, легко нахожу контакт с собеседником Работоспособность, коммуникабельность, целеустремленность, ответственность, исполнительность, самостоятельность, стрессоустойчивость, активная жизненная позиция. Активна, амбициозна, быстрая обучаемость, вежливая, внимательная, добросовестная, ответственная. аналитические способности; внимательность; математический склад ума; инициативность; логическое мышление; ответственность; уверенность; усидчивость; честность. - объективна, энергична, коммуникабельна - нацелена на результат, инициативна  исполнительна,с высоким чувством ответственности - самостоятельна, умею работать в команде - легко осваиваюсь - интересно любое направление и сфера деятельности Аккуратность, исполнительность, коммуникабельность, активность. В последние годы проходил обучение без возможности работать Веду образ жизни'
stop_words = set(tokenize(bad_words))
stop_words.add("'\\")
stop_words.add("\\")
stop_words.add("\n")

In [ ]:
tokenize_skills(resumes0.skills[1424790])

In [ ]:
%%time

for pa_name, pa_translit, use_skill in zip(pa_names, pa_translits, use_skills):
#     if pa_name == 'Информационные технологии, интернет, телеком':
#         continue
        
    if use_skill:
        resumes = resumes0[(resumes0.professional_area == pa_name) & resumes0.position.notna() & resumes0.skills.notna() & (resumes0.skills.str.len() > 2)]
    else:
        resumes = resumes0[(resumes0.professional_area == pa_name) & resumes0.position.notna()]
    resumes = resumes.sample(n=(resumes.shape[0] if resumes.shape[0] < 100000 else 100000), random_state=42)
    print(pa_name, resumes.shape)
    
    #area_id
    areas = resumes.groupby(['area_id']).size().apply(lambda x: math.log(x+1))
    resumes['area_id'] = resumes['area_id'].apply(lambda x: areas.loc[x])
#     resumes['area_id'].plot.hist(bins=200)
#     plt.show()
    
    #gender
    resumes['gender_bin'] = resumes['gender'].replace(r"'\N'", 'male').apply(lambda x: 1 if x == 'male' else 0)
#     resumes['gender_bin'].plot.hist()
#     plt.show()
    
    #birth
    age_mo = (pd.datetime.now()-pd.to_datetime(resumes.birth_day, errors='coerce'))/np.timedelta64(1, 'M')
    resumes['age_mo'] = age_mo
    prof_area_age_mean = resumes.groupby(by='professional_area').age_mo.mean()
    spec_age_mean = resumes.groupby(by='spec_name').age_mo.mean()

    for i, row in resumes.iterrows():
        if math.isnan(row['age_mo']):
            resumes.at[i, 'age_mo'] = spec_age_mean.loc[row['spec_name']] if (not math.isnan(spec_age_mean.loc[row['spec_name']])) else prof_area_age_mean.loc[row['professional_area']]

    resumes['age_mo'] = resumes.age_mo.apply(lambda x: math.log(x))
#     resumes['age_mo'].plot.hist(bins=200)
#     plt.show()

    #education
    lb = preprocessing.LabelBinarizer()
    education_level = lb.fit_transform(resumes['education_level'])
    education_level = pd.DataFrame(education_level).drop(columns=[2])
    education_level = education_level.drop(columns=education_level.var()[education_level.var() < 0.005].index.values)

    #position
    vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize, min_df=5, max_df=0.5, use_idf=1, stop_words=stop_words)
    best_position = vec.fit_transform(resumes.position)
    print('Position', best_position.shape)
    
    #compensation
    rates = {
        'AZN': 0.027442,
        'BYR': 0.032,
        'EUR': 0.01363,
        'GEL': 0.0344,
        'KGS': 1.096834,
        'KZT': 5.364433,
        'RUR': 1.0,
        'UAH': 0.419292,
        'USD': 0.01604,
        'UZS': 127.424733,    
    }
    resumes['zp'] = (resumes.compensation/resumes.currency.map(rates))
    prof_area_mean = resumes.groupby(by='professional_area').zp.mean()
    spec_mean = resumes.groupby(by='spec_name').zp.mean()
    for i, row in resumes.iterrows():
        if math.isnan(row['zp']) or row['zp'] < 100.0:
            resumes.at[i, 'zp'] = spec_mean.loc[row['spec_name']] if (not math.isnan(spec_mean.loc[row['spec_name']]) and spec_mean.loc[row['spec_name']] > 100) else prof_area_mean.loc[row['professional_area']]

    resumes.zp = resumes.zp.apply(lambda x: math.log(x))
#     resumes.zp.plot.hist(bins=200) 
    
    #work schedule
    resumes.work_schedule = resumes.work_schedule.replace(r"'\N'", 'full_day')
    resumes.work_schedule.value_counts()
    lb = preprocessing.LabelBinarizer()
    work_schedule = pd.DataFrame(lb.fit_transform(resumes['work_schedule'])).rename(lambda x: 'work_schedule_enc_' + str(x), axis=1)
    
    #employment
    m = {
        'full': 1,  
        'part': 0, 
        'project': 0,
        'probation': 0,
        'volonteer': 0
    }
    resumes['employment_m'] = resumes.employment.map(m)
    resumes.employment_m.value_counts()
    
    #language_level
    languages = ['kir','mya','udm','aze','sqi','ara','hye','afr','bak','bel','bul','hun','vie','nld','ell','kat','dan','heb','spa','ita','kaz','zho','kom','kor','kur','lav','lit','mke','mon','nor','fas','pol','por','ron','rus','san','srp','slk','slv','swa','tgk','tha','tat','tur','tuk','uzb','ukr','urd','fin','hrv','che','ces','chv','swe','est','jpn','hin','eng','deu','fra','ind','inh','kas','epo','khm','crs','abk','oss','kbd','krc','cat','uig','abq','kum','tam','lat','zlm','nog','amh','dar','ava','eus','ben','bos','bua','dag','gle','isl','krl','lbe','lao','lez','chm','nep','pan','pus','som','tgl','tly','bod']
    res_languages = []
    for l in languages:
        if resumes[l].value_counts()[0] < resumes.shape[0]*0.995:
            res_languages.append(l)

    lang_map = {
        'abs': 0, #тут подзабил на то, что в эту группу попали те, кто не владеет и те, у кого не указано
        'basic': 1,
        'can_read': 2,
        'can_pass_interview': 3,
        'fluent': 4,
        'native': 5
    }

    languages_res = pd.DataFrame(index=resumes.index)
    for lng in res_languages:
        languages_res[lng] = resumes[lng].fillna('abs').apply(lambda x: 'abs' if x == 'none' else x).map(lang_map)
        
    #skills
    if use_skill:
        vec = TfidfVectorizer(ngram_range=(1,1), tokenizer=tokenize_skills, min_df=8, max_df=0.5, use_idf=1, stop_words=stop_words, max_features=10000)
        best_skills = vec.fit_transform(resumes.skills)
        print('Skills', best_skills.shape)
    
    #spec
    cnt_v = CountVectorizer(tokenizer=tokenize_spec)
    specializations = cnt_v.fit_transform(resumes.spec_name)
    sp = pd.DataFrame(specializations.toarray())
    sp_var = sp.var()
    sp = sp.drop(columns=sp_var[sp_var < 0.00002].index.values)
    
    #work_experience_month_count
    resumes.work_experience_month_count = resumes.work_experience_month_count.apply(lambda x: math.log(x+1))
    
    #### COMBINE ####
    if use_skill:
        X_sparse = sparse.hstack((best_position, best_skills, csr_matrix(languages_res), csr_matrix(sp), csr_matrix(education_level), csr_matrix(work_schedule), csr_matrix(resumes[['zp', 'work_experience_month_count', 'area_id', 'gender_bin', 'age_mo', 'employment_m']])))
    else:
        X_sparse = sparse.hstack((best_position, csr_matrix(languages_res), csr_matrix(sp), csr_matrix(education_level), csr_matrix(work_schedule), csr_matrix(resumes[['zp', 'work_experience_month_count', 'area_id', 'gender_bin', 'age_mo', 'employment_m']])))
    
    print(X_sparse.shape)
    
    norm = StandardScaler(with_mean=True)
    X_sparse = X_sparse.toarray()
    print('to array')
    X_sparse = csr_matrix(norm.fit_transform(X_sparse))
    print('Scaled', X_sparse.shape)
    svd = TruncatedSVD(n_components=100, random_state=42)
    X_sparse_svd = svd.fit_transform(X_sparse)
    norm = Normalizer()
    X_norm = norm.fit_transform(X_sparse_svd)
    X = pd.DataFrame(X_norm, index=resumes.index)
    X = X.rename(lambda x: 'f0_' + str(x), axis=1)
    
    X.to_csv('/home/shurik2533/res_X/X_{}.csv'.format(pa_translit))

    X_sample = X.sample(n=(X.shape[0] if X.shape[0] < 30000 else 30000), random_state=42)
    resumes_sample = resumes.loc[X_sample.index.values]
    X_sample.to_csv('/home/shurik2533/res_X/X_sample_{}.csv'.format(pa_translit))
    resumes_sample.to_csv('/home/shurik2533/res_X/resumes_sample_{}.csv'.format(pa_translit))
    print('Start tsne', pa_name)
    X_embedded = TSNE(n_components=2, random_state=42).fit_transform(X_sample)
    print('End tsne', pa_name)
    
    tdf = pd.DataFrame(resumes['spec_name_hash'], index=X_sample.index)
    tdf['x'] = X_embedded[:,0]
    tdf['y'] = X_embedded[:,1]
    tdf['resume_id'] = resumes['resume_id']
    tdf['title'] = resumes['position']
    tdf['spec_name'] = resumes['spec_name']
    tdf.to_csv('/home/shurik2533/res_X/for_vis_{}.csv'.format(pa_translit))
    plt.ioff()
    plt.figure(figsize=(15, 15))
    l = list(tdf.spec_name_hash.unique())
    plt.scatter(tdf.x, tdf.y, cmap='prism', s=1.5) 
    plt.savefig('/home/shurik2533/res_X/img/{}'.format(pa_translit), bbox_inches='tight')
    plt.close()
    